In [1]:
!pip install pymongo

In [11]:
from pymongo import MongoClient
import pandas as pd
import json

In [3]:
db_name = "fleximart"
collection = "products_catalog"
mongo_url = "mongodb://localhost:27017/"

In [7]:
with open(file="product_catalog.json", mode="r", encoding="utf-8") as f:
    data = json.load(f)

In [8]:
client = MongoClient(mongo_url)
db = client[db_name]
col = db[collection]

Operation 1: Load Data

In [9]:
col.delete_many({})
rec = col.insert_many(data)
print(f'Inserted {len(rec.inserted_ids)} documents into {db_name}.{collection}')


Inserted 12 documents into fleximart.products_catalog


Operation 2: Basic Query

In [12]:
q2 = col.find(
    {"category": "Electronics", "price": {"$lt": 50000}},
    {"_id": 0, "name": 1, "price": 1, "stock": 1}
)

pd.DataFrame(list(q2))

,name,price,stock
0,Sony WH-1000XM5 Headphones,29990.0,200
1,Dell 27-inch 4K Monitor,32999.0,60
2,OnePlus Nord CE 3,26999.0,180


Operation 3: Review Analysis

In [14]:
pipeline3 = [
    {
        "$addFields": {
            "avg_rating": {
                "$avg": {"$ifNull": ["$reviews.rating", []]}
            }
        }
    },
    {
        "$match": {"avg_rating": {"$gte": 4.0}}
    },
    {
        "$project": {
            "_id": 0,
            "product_id": 1,
            "name": 1,
            "category": 1,
            "avg_rating": {"$round": ["$avg_rating", 2]}
        }
    }
]

pd.DataFrame(list(col.aggregate(pipeline3)))

,product_id,name,category,avg_rating
0,ELEC001,Samsung Galaxy S21 Ultra,Electronics,4.67
1,ELEC002,Apple MacBook Pro 14-inch,Electronics,5.00
2,ELEC003,Sony WH-1000XM5 Headphones,Electronics,4.67
3,ELEC004,Dell 27-inch 4K Monitor,Electronics,4.00
4,ELEC005,OnePlus Nord CE 3,Electronics,4.00
5,ELEC006,Samsung 55-inch QLED TV,Electronics,4.50
6,FASH001,Levi's 511 Slim Fit Jeans,Fashion,4.67
7,FASH002,Nike Air Max 270 Sneakers,Fashion,4.50
8,FASH003,Adidas Originals T-Shirt,Fashion,4.33
9,FASH004,Puma RS-X Sneakers,Fashion,4.50


Operation 4: Update Operation

In [15]:
import datetime

In [16]:
new_review = {
    "user": "U999",
    "rating": 4,
    "comment": "Good value",
    "date": datetime.datetime.now()  
}

In [17]:
res4 = col.update_one(
    {"product_id": "ELEC001"},
    {"$push": {"reviews": new_review}}
)

print("matched:", res4.matched_count, "modified:", res4.modified_count)

matched: 1 modified: 1


In [18]:
updated = col.find_one(
    {"product_id": "ELEC001"},
    {"_id": 0, "product_id": 1, "name": 1, "reviews": 1}
)

updated

{'product_id': 'ELEC001',
 'name': 'Samsung Galaxy S21 Ultra',
 'reviews': [{'user_id': 'U001',
   'username': 'TechGuru',
   'rating': 5,
   'comment': 'Excellent phone with amazing camera quality!',
   'date': '2024-01-15'},
  {'user_id': 'U012',
   'username': 'MobileUser',
   'rating': 4,
   'comment': 'Great performance but a bit pricey.',
   'date': '2024-02-10'},
  {'user_id': 'U023',
   'username': 'PhotoEnthusiast',
   'rating': 5,
   'comment': "Best camera phone I've ever used!",
   'date': '2024-03-05'},
  {'user': 'U999',
   'rating': 4,
   'comment': 'Good value',
   'date': datetime.datetime(2026, 1, 7, 22, 32, 25, 204000)}]}

Operation 5: Complex Aggregation


In [19]:
pipeline5 = [
    {
        "$group": {
            "_id": "$category",
            "avg_price": {"$avg": "$price"},
            "product_count": {"$sum": 1}
        }
    },
    {
        "$project": {
            "_id": 0,
            "category": "$_id",
            "avg_price": {"$round": ["$avg_price", 2]},
            "product_count": 1
        }
    },
    {
        "$sort": {"avg_price": -1}
    }
]

pd.DataFrame(list(col.aggregate(pipeline5)))

,product_count,category,avg_price
0,6,Electronics,70830.83
1,6,Fashion,5215.00
